# Импортируем ДФ

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# выгрузим первые 5 строк каждой таблицы
queries = [
    '''SELECT * FROM books LIMIT 5''',
    '''SELECT * FROM authors LIMIT 5''',
    '''SELECT * FROM publishers LIMIT 5''',
    '''SELECT * FROM ratings LIMIT 5''',
    '''SELECT * FROM reviews LIMIT 5'''
]

# выполним запросы и сохраним результаты в ДФ
table_names = ['books', 'authors', 'publishers', 'ratings', 'reviews']
dataframes = {}
with engine.connect() as con:
    for query, table_name in zip(queries, table_names):
        result = con.execute(text(query))
        data = result.fetchall()
        columns = result.keys()
        df = pd.DataFrame(data, columns=columns)
        dataframes[table_name] = df

for table_name, df in dataframes.items():
    print(f"Информация о таблице '{table_name}':")
    print(df.info())
    print("\n")
    print("\nПервые 5 строк таблицы:")
    print(df.head())
    print("\n")
    print("\n")

Информация о таблице 'books':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes
None



Первые 5 строк таблицы:
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125              

## Задачи

### сколько книг вышло после 1 января 2000 года;

In [3]:
query = '''
SELECT COUNT(*) AS num_books
FROM books
WHERE publication_date > '2000-01-01'
'''
with engine.connect() as con:
    result = con.execute(text(query))
    num_books = result.fetchone()[0]

print(f"кол-во книг, вышедших после 1 января 2000 года: {num_books}")

кол-во книг, вышедших после 1 января 2000 года: 819


### для каждой книги посчитаем количество обзоров и среднюю оценку

In [4]:
query_rev = '''                              
    SELECT
        books.title AS book_title,
        books.book_id AS book_id,
        AVG(ratings.rating) AS avg_rating,
        COUNT(DISTINCT reviews.review_id) AS count_reviews
    FROM
        books
    INNER JOIN ratings ON ratings.book_id = books.book_id
    LEFT JOIN reviews ON reviews.book_id = books.book_id
    GROUP BY
        books.title,
        books.book_id
    ORDER BY
        count_reviews DESC;
'''

with engine.connect() as con:
    df_rev = pd.io.sql.read_sql(sql=text(query_rev), con=con)

df_rev['avg_rating'] = df_rev['avg_rating'].round(2)
df_rev


,book_title,book_id,avg_rating,count_reviews
0,Twilight (Twilight #1),948,3.66,7
1,Water for Elephants,963,3.98,6
2,The Glass Castle,734,4.21,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.41,6
4,The Curious Incident of the Dog in the Night-Time,695,4.08,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.67,0
996,The Natural Way to Draw,808,3.00,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.00,0
998,Essential Tales and Poems,221,4.00,0


In [5]:
sum_count_reviews = df_rev['count_reviews'].sum()
sum_count_reviews

2793

### определим издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [6]:
query = '''
SELECT p.publisher, COUNT(b.book_id) AS num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY num_books DESC
LIMIT 1
'''
with engine.connect() as con:
    result = con.execute(text(query))
    data = result.fetchone()


publisher = data[0]
num_books = data[1]
print(f"издательство, которое выпустило наибольшее число книг (толще 50 страниц): {publisher}")
print(f"кол-во книг: {num_books}")

издательство, которое выпустило наибольшее число книг (толще 50 страниц): Penguin Books
кол-во книг: 42


### автор с самой высокой средней оценкой книг

In [7]:
query = '''
SELECT a.author, AVG(r.rating) AS avg_rating
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN (
    SELECT book_id, COUNT(rating) AS num_ratings
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating) >= 50
) sub ON b.book_id = sub.book_id
JOIN ratings r ON b.book_id = r.book_id
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1
'''
with engine.connect() as con:
    result = con.execute(text(query))
    data = result.fetchone()

author = data[0]
avg_rating = data[1]
print(f"автор с самой высокой средней оценкой книг: {author}")
print(f"средняя оценка: {avg_rating:.2f}")


автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré
средняя оценка: 4.29


### среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [8]:
query = '''
SELECT AVG(num_reviews) AS avg_reviews
FROM (
    SELECT username, COUNT(review_id) AS num_reviews
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating) > 48
    )
    GROUP BY username
) sub
'''

with engine.connect() as con:
    result = con.execute(text(query))
    avg_reviews = result.fetchone()[0]

print(f"среднее количество обзоров от пользователей, которые поставили больше 48 оценок: {avg_reviews:.2f}")

среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.00


## Выводы

1. после 1 января 2000 года вышло 819 книги  
2. рассчитали среднюю оценку для каждоый книги. 
3. издательство Penguin Books выпустило наибольшее число книг (толще 50 страниц): 42 шт.  
4. автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré (средняя оценка: 4.29)  
5. среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24